Trabajaremos con el archivo **final_consumo_agua** que se obtiene después de realizar el análisis exploratorio de datos.

In [ ]:
%run main_file.py

In [ ]:
final_consumo_agua.head()

# Feature Selection 

In [ ]:
import pandas as pd
import feature_selection

Definimos las variables numéricas y categóricas de interés:

In [ ]:
numeric_variables = ['consumo_total_mixto', 'consumo_prom_dom',
       'consumo_total_dom', 'consumo_prom_mixto', 'consumo_prom',
       'consumo_prom_no_dom', 'consumo_total_no_dom', 'latitud', 'longitud']
string_variables = ['alcaldia', 'indice_des','bimestre']

Dividimos nuestro dataframe en la variable a predecir **y_outcome** y las variables que nos ayudaran en el modelo **x_regressors**.

In [ ]:
y_output = final_consumo_agua['consumo_total']
x_regressors = pd.concat([final_consumo_agua[numeric_variables], final_consumo_agua[string_variables]], axis = 1)

Imputamos y estandarizamos las variables numéricas

In [ ]:
imputed_vars = feature_selection.mean_imputation_for_numeric_vars(x_regressors, numeric_variables)

standardized_vars = feature_selection.standardization_for_numeric_vars(imputed_vars)

# Asignamos nuevamente al dataframe de interés
x_regressors.loc[:, numeric_variables] = standardized_vars

Hacemos el *hot encoding* para las variables categoricas

In [ ]:
dummies = feature_selection.transforma_cat_dummies(x_regressors, string_variables, True)

Eliminamos las variables categoricas y añadimos las nuevas variables dummies

In [ ]:
x_regressors = pd.concat([x_regressors, dummies], axis = 1)

x_regressors = x_regressors.drop(string_variables, axis = 1)

print("Se tiene 1 constante mas {} variables: {} ".format(x_regressors.shape[1], x_regressors.columns))

## Backward selection

Definimos el valor de corte para el p-value

In [ ]:
P_VALUE_THRESHOLD = 0.05

In [ ]:
selected_vars = feature_selection.backward_selection(y_output, x_regressors, P_VALUE_THRESHOLD)

In [ ]:
print("Se tienen {} variables -incluyendo la cte: {} ".format(selected_vars.shape[1], selected_vars.columns))

## Chi-squared selection

Realizamos el one hot encoding para categoricas (dummies)

In [ ]:
categoricas_dummies = feature_selection.transforma_cat_dummies(final_consumo_agua, ['alcaldia','indice_des'], False)

Convertimos la variable consumo en una variable binaria

In [ ]:
consumo_alto = feature_selection.transforma_num_a_binaria(final_consumo_agua,'consumo_total','consumo_alto',0.75)

Realizamos la prueba chi-cuadrada para las variables categoricas

In [ ]:
prueba_chi = feature_selection.chi_squared(categoricas_dummies, consumo_alto)

Filtramos las variables de las cuales rechazamos independencia, mismas que se utilizaran como variables predictivas

In [ ]:
vars_seleccionadas = prueba_chi[prueba_chi['significativo'] == 'no significativo']['var_cat_predictivas']
print(vars_seleccionadas)